In [2]:
include("../src/Juissie.jl")
using .Juissie

### Initializing a Generator
Generators are how we will generate text for a given query (and contextual information from our vector DB).

Here, we'll initialize an `OAIGenerator`, which calls OpenAI's
gpt-3.5-turbo completion endpoint. If you have an OpenAI API key you can do one of two things:
1. Pass it directly to `OAIGenerator`, e.g. `generator = OAIGenerator("YOUR_KEY_HERE")`. If you do this, though, make sure you don't accidentally commit your key!
2. Leave the `auth_token` argument as `nothing`. When you do this, `OAIGenerator` will look in your environmental variables for a key called "OAI_KEY".

We'll be doing the second option:

In [2]:
generator = OAIGenerator();

In [3]:
result = generate(
    generator,
    "Count to 100 by increments of x", # this is the main query, i.e. your question
    ["x=9"] # this is the context we will provide (chunks from the vector db)
)

"9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99"

### Generating With A Corpus

We're going to create a generator of type OAIGeneratorWithCorpus, which is just like OAIGenerator, except it also has a corpus attached. The usual upsert functions one might apply to a corpus have equivalents for structs that subtype GeneratorWithCorpus, as well, such as:
- upsert_chunk_to_generator
- upsert_document_to_generator
- upsert_document_from_url_to_generator

We'll fill the generator's corpus with chunks from the Wikipedia article on Aristotle, then ask the generator a niche question whose answer is found in that article.

In [4]:
generator = OAIGeneratorWithCorpus();

In [3]:
upsert_document_from_url_to_generator(
    generator, 
    "https://en.wikipedia.org/wiki/Aristotle", 
    "Wikipedia: Aristotle"
)

In [5]:
result, idx_list, doc_names, chunks = generate_with_corpus(
    generator,
    "According to Aristotle, what are the six elements of which tragedy is composed?",
    3,
    0.7
);

println(result, idx_list)

Tragedy, according to Aristotle, is composed of six elements: plot-structure, character, style, thought, spectacle, and lyric poetry.[22, 23, 4]


While not necessary, it would be really cool if our top-retrieved context result is the exact section that has the answer (this is indeed the case - see the fourth to last sentence):

In [8]:
println(chunks[1])

[131]  Aristotle also outlines two kinds of rhetorical  proofs :  enthymeme  (proof by  syllogism ) and  paradeigma  (proof by example). [132] Aristotle writes in his  Poetics  that  epic poetry , tragedy, comedy,  dithyrambic poetry , painting, sculpture, music, and dance are all fundamentally acts of  mimesis  ("imitation"), each varying in imitation by medium, object, and manner. [133] [134]  He applies the term  mimesis  both as a property of a work of art and also as the product of the artist's intention [133]  and contends that the audience's realisation of the  mimesis  is vital to understanding the work itself. [133]  Aristotle states that  mimesis  is a natural instinct of humanity that separates humans from animals [133] [135]  and that all human artistry "follows the pattern of nature". [133]  Because of this, Aristotle believed that each of the mimetic arts possesses what  Stephen Halliwell  calls "highly structured procedures for the achievement of their purposes." [136]  

### More Involved Example: Philosophers
Here, we will create an on-disk corpus and fill it with the articles about Greek philosophers.

In [3]:
generator = OAIGeneratorWithCorpus("greek_philosophers");

┌ Warning: A corpus by this name already exists. Do you want to proceed? Doing so will overwrite the existing corpus artifacts. [y/n]
└ @ Main.Juissie.Generation.SemanticSearch.Backend ~/Desktop/Juissie.jl/src/SemanticSearch/Backend.jl:113


stdin>  y


In [4]:
greek_philosophers = Dict(
    "Wikipedia: Aristotle" => "https://en.wikipedia.org/wiki/Aristotle",
    "Wikipedia: Democrates" => "https://en.wikipedia.org/wiki/Democrates",
    "Wikipedia: Diogenes" => "https://en.wikipedia.org/wiki/Diogenes_Laertius",
    "Wikipedia: Epictetus" => "https://en.wikipedia.org/wiki/Epictetus",
    "Wikipedia: Epicurus" => "https://en.wikipedia.org/wiki/Epicurus",
    "Wikipedia: Heraclitus" => "https://en.wikipedia.org/wiki/Heraclitus",
    "Wikipedia: Parmenides" => "https://en.wikipedia.org/wiki/Parmenides",
    "Wikipedia: Plato" => "https://en.wikipedia.org/wiki/Plato",
    "Wikipedia: Socrates" => "https://en.wikipedia.org/wiki/Socrates",
    "Wikipedia: Xenophon" => "https://en.wikipedia.org/wiki/Xenophon",
    "Wikipedia: Ancient Greek philosophy" => "https://en.wikipedia.org/wiki/Ancient_Greek_philosophy",
    "Internet Encyclopedia of Philosophy: Ancient Greek Philosophy" => "https://iep.utm.edu/ancient-greek-philosophy/",
    "Stanford Encyclopedia of Philosophy: Presocratic Philosophy" => "https://plato.stanford.edu/entries/presocratics/",
    "Stanford Encyclopedia of Philosophy: Ancient Political Philosophy" => "https://plato.stanford.edu/entries/ancient-political/",
    "Stanford Encyclopedia of Philosophy: Aristotle’s Political Theory" => "https://plato.stanford.edu/entries/aristotle-politics/",
    "Stanford Encyclopedia of Philosophy: Anaxagoras" => "https://plato.stanford.edu/entries/anaxagoras/",
    "Stanford Encyclopedia of Philosophy: Heraclitus" => "https://plato.stanford.edu/entries/heraclitus/",
    "Stanford Encyclopedia of Philosophy: Pythagoras" => "https://plato.stanford.edu/entries/pythagoras/",
    "Stanford Encyclopedia of Philosophy: Ancient Ethical Theory" => "https://plato.stanford.edu/entries/ethics-ancient/",
    "Stanford Encyclopedia of Philosophy: Theophrastus" => "https://plato.stanford.edu/entries/theophrastus/",
    "Stanford Encyclopedia of Philosophy: Zeno’s Paradoxes" => "https://seop.illc.uva.nl/entries/paradox-zeno/",
    "Stanford Encyclopedia of Philosophy: The Sophists" => "https://plato.stanford.edu/entries/sophists/",
    "Stanford Encyclopedia of Philosophy: Protagoras" => "https://plato.stanford.edu/entries/protagoras/",
    "Stanford Encyclopedia of Philosophy: Parmenides" => "https://plato.stanford.edu/entries/parmenides/",
    "Stanford Encyclopedia of Philosophy: Empedocles" => "https://plato.stanford.edu/entries/empedocles/",
);
for (key, value) in greek_philosophers
    upsert_document_from_url_to_generator(
        generator, 
        value, 
        key
    )
end

In [5]:
result, idx_list, doc_names, chunks = generate_with_corpus(
    generator,
    "Contrast the lives of Anaxagoras and Empedocles.",
    10,
    0.7
);

println(result, idx_list)

Anaxagoras and Empedocles both delved into the fundamental elements of the cosmos, but their approaches and philosophies diverged significantly. Anaxagoras, hailing from Clazomenae, introduced the concept of "everything-in-everything" and attributed the cosmos' creation to "nous," or intellect. He was renowned for explaining eclipses and posited radical scientific theories about celestial bodies. In contrast, Empedocles, a native of Acragas in Sicily, championed a four-part theory of roots (earth, water, air, fire) and incorporated the forces of Love and Strife into his cosmology. While Empedocles' view on Love and Strife was more intricate, involving both mixing and separating, Anaxagoras focused on the dynamic motion of ingredients to form the diverse material world we observe. Furthermore, Empedocles' philosophical musings intertwined with mysticism, religion, and medicine, placing him at the crossroads of myth and reason, while Anaxagoras paved the way for a scientific understandin

We can also load a *new* generator from the `greek_philosophers` artifacts and query *that*.

In [2]:
generator = load_OAIGeneratorWithCorpus("scifi");

In [4]:
generator.corpus.next_idx

66